In [1]:
import numpy as np
import pandas 

df = pandas.read_csv("https://raw.githubusercontent.com/ChristopheHunt/MSDA"+ 
                     "---Coursework/master/Data%20602/Data%20602%20-%20" + 
                     "Advanced%20Programming/Week%2011/apple.2011.csv")

df.columns = [ 'Date', 'Last', 'Change']
df['Change'] = pandas.to_numeric(df['Change'], errors='coerce')

In [2]:
def VaR(n, days):
    import pandas
    list_values = []
    mu, sigma = df['Change'].mean(), df['Change'].std()
    for j in range(1,n):
        random_numbers = np.random.normal(mu, sigma, days)
        last_price = df['Last'].tail(1)
        last_price = last_price.iloc[0]
        for i in random_numbers:
            last_price =  (i * last_price) + last_price 
        list_values.append(last_price)
        dataframe = pandas.DataFrame(list_values)
        dataframe.columns = ['values']
    return(df)

In [3]:
days = 20 
n = 1000
VaR_df = VaR(n, days)
lowest = VaR_df.quantile(q = .01).iloc[0]

In [4]:
import timeit
number = 5
setup = ("from __main__ import VaR, df, n, days")
results = timeit.timeit("VaR_df = VaR(n, days); lowest = VaR_df.quantile(q = .01).iloc[0]", setup=setup, number = number)
print "The Monte Carlo solution without parallel processing is : " + format(results,'.8f') + " for " + str(number) + " loops"

The Monte Carlo solution without parallel processing is : 2.25956135 for 5 loops


# Parallel Processing

The following processes the list in each environment

In [6]:
#ipcluster start -n 4
import ipyparallel as ipp 
c = ipp.Client()
view = c.load_balanced_view()

<LoadBalancedView None>


In [ ]:
%%px
import numpy as np
import pandas 

df = pandas.read_csv("https://raw.githubusercontent.com/ChristopheHunt/MSDA"+ 
                     "---Coursework/master/Data%20602/Data%20602%20-%20" + 
                     "Advanced%20Programming/Week%2011/apple.2011.csv")

df.columns = [ 'Date', 'Last', 'Change']
df['Change'] = pandas.to_numeric(df['Change'], errors='coerce')

def VaR(n, days):
    import pandas
    list_values = []
    mu, sigma = df['Change'].mean(), df['Change'].std()
    for j in range(1,n):
        random_numbers = np.random.normal(mu, sigma, days)
        last_price = df['Last'].tail(1)
        last_price = last_price.iloc[0]
        for i in random_numbers:
            last_price =  (i * last_price) + last_price 
        list_values.append(last_price)
        dataframe = pandas.DataFrame(list_values)
        dataframe.columns = ['values']
    return(df)

days = 20 
n = 1000

import timeit
number = 5

setup = ("from __main__ import VaR, df, n, days")
results = timeit.timeit("VaR_df = VaR(n, days); lowest = VaR_df.quantile(q = .01).iloc[0]", setup=setup, number = number)
print "The Monte Carlo solution with parallel processing is : " + format(results,'.8f') + " for " + str(number) + " loops"

# Parallel Processing 

This following code is my attempt at processing the variance in the environments in sections. This is partioning the work into 4 sections and combine the results. 

In [ ]:
%%px
import numpy as np
import pandas 

import ipyparallel as ipp 
c = ipp.Client()
v = c[:]

df = pandas.read_csv("https://raw.githubusercontent.com/ChristopheHunt/MSDA"+ 
                     "---Coursework/master/Data%20602/Data%20602%20-%20" + 
                     "Advanced%20Programming/Week%2011/apple.2011.csv")

df.columns = [ 'Date', 'Last', 'Change']
df['Change'] = pandas.to_numeric(df['Change'], errors='coerce')

def VaR(n, days):
    import pandas
    list_values = []
    mu, sigma = df['Change'].mean(), df['Change'].std()
    for j in range(1,n):
        random_numbers = np.random.normal(mu, sigma, days)
        last_price = df['Last'].tail(1)
        last_price = last_price.iloc[0]
        for i in random_numbers:
            last_price =  (i * last_price) + last_price 
        list_values.append(last_price)
    list_values
    return(list_values)

days = 20 
n = 20

y = VaR(n,days)
complete_list = v.map('y')
complete_list